In [1]:
import html
import re

import tensorflow_datasets as tfds

class SubalphabetSubwordTextEncoder:
    HTML_ESCAPE_PATTERN = re.compile('&(((#\d+)|([A-Za-z]+));&?)+')
    SPECIAL_CHAR = 'X'

    def __init__(self, subalphabet, vocab_list=None):
        self._subword_text_encoder = tfds.features.text.SubwordTextEncoder(vocab_list)
        self._subalphabet = subalphabet

    def encode(self, s):
        subword_max_code = len(self.subwords)
        codes = self._build_encoding_dict(self._subalphabet)
        preprocessed_string = self.preprocess_string(s)

        result = []
        unicode_buffer = bytearray()
        for id in self._subword_text_encoder.encode(preprocessed_string):
            if id <= subword_max_code:
                result.append(id)
            else:
                unicode_buffer.append(id - subword_max_code - 1)
                try:
                    id = ord(unicode_buffer.decode())
                    result.append(subword_max_code + codes[id] + 1)
                    unicode_buffer = bytearray()
                except UnicodeDecodeError:
                    pass
        return result

    def decode(self, ids):
        subword_max_code = len(self.subwords)

        processed_ids = []
        for id in ids:
            if id <= subword_max_code:
                processed_ids.append(id)
            else:
                char = self._subalphabet[id - subword_max_code - 1].encode()
                processed_ids += [codepoint + subword_max_code + 1 for codepoint in char]

        return self._subword_text_encoder.decode(processed_ids)
    
    @classmethod
    def build_from_corpus(cls, corpus_generator, subalphabet_size, target_vocab_size, max_subword_length=20, max_corpus_chars=None, reserved_tokens=None):
        simplified_strings = []
        char_counts = defaultdict(lambda: 0)

        for string in corpus_generator:
            simplified_string = cls._unescape_string(string.lower())
            simplified_strings.append(simplified_string)
            for char in simplified_string:
                char_counts[char] += 1

        sorted_char_counts = sorted(char_counts.items(), key = lambda item: item[1], reverse = True)
        chars = [char for char, char_count in sorted_char_counts[:subalphabet_size - 1]]
        chars = list(reversed(chars)) # We reverse the chars so that the most common ones are last
        chars = cls._ensure_special_character_presence(chars)

        def simplified_corpus():
            for string in simplified_strings:
                yield cls._compact_string(chars, string)

        subword_text_encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(simplified_corpus(), target_vocab_size + 256 - subalphabet_size)
        
        return cls(chars, subword_text_encoder.subwords)
    
    @classmethod
    def load_from_file(cls, filename_prefix):
        subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file(filename_prefix)
        with open(filename_prefix + '.subalphabet', 'rb') as text_file:
            subalphabet = ['\0', cls.SPECIAL_CHAR] + text_file.read().decode().split('\0')

        return cls(subalphabet, subword_text_encoder.subwords)
    
    def save_to_file(self, filename_prefix):
        self._subword_text_encoder.save_to_file(filename_prefix)
        with open(filename_prefix + '.subalphabet', 'wb') as text_file:
            text_file.write('\0'.join(self._subalphabet[2:]).encode())

    def preprocess_string(self, string):
        return self._compact_string(self._subalphabet, self._unescape_string(string.lower()))

    @property
    def vocab_size(self):
        return self._subword_text_encoder.vocab_size - 256 + len(self._subalphabet)
    
    @property
    def subwords(self):
        return self._subword_text_encoder.subwords

    @classmethod
    def _ensure_special_character_presence(cls, chars):
        if '\0' in chars:
            chars = [char for char in chars if char != '\0']
        else:
            chars = chars[1:]

        return ['\0', cls.SPECIAL_CHAR] + chars
    
    @classmethod
    def _unescape_html_symbol(cls, escaped):
        escaped_symbols = escaped.group(0).split(';')
        escaped_symbols = escaped_symbols[:-1] # strip the last string empty string
        escaped_symbols = (escaped_symbol + ';' for escaped_symbol in escaped_symbols)
        result = ''

        for escaped_symbol in escaped_symbols:
            code = escaped_symbol[1:] if escaped_symbol[0] == '&' else escaped_symbol

            if code[0] == '#':
                result += chr(int(code[1:-1]))
            elif code in html.entities.html5:
                result += html.entities.html5[code]
            else:
                result += escaped_symbol

        return result
    
    @classmethod
    def _unescape_string(cls, string):
        return cls.HTML_ESCAPE_PATTERN.sub(cls._unescape_html_symbol, string)
    
    @classmethod
    def _compact_string(cls, chars, string):
        if len(string) == 0:
            return string
        else:
            compacted_string = string[0]

            for char in string[1:]:
                if char not in chars or char == cls.SPECIAL_CHAR:
                    if compacted_string[-1] != cls.SPECIAL_CHAR:
                        compacted_string += cls.SPECIAL_CHAR
                else:
                    compacted_string += char

            return compacted_string
    
    @classmethod
    def _build_encoding_dict(cls, chars):
        return {ord(char): index for index, char in enumerate(chars)}

In [2]:
encoder = SubalphabetSubwordTextEncoder.load_from_file('72_256')

In [3]:
import itertools
import math
import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from checkpointing import checkpointable

physical_devices = tf.config.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, enable=True)

In [4]:
TYPE=np.int16

class Articles:
    EMPTY_ARTICLE = np.array([], dtype=TYPE) # used for padding
    
    def __init__(self, path):
        with open(path, 'rb') as text_file:
            data = text_file.read()[:100000000].decode()

        self.articles = sorted(set(data.split('\0')), key=len)
        self._encoded_articles = None

    @property
    def encoded_articles(self):
        if self._encoded_articles == None:
            self._encoded_articles = [np.array(encoder.encode(article), dtype=TYPE) for article in self.articles]
        
        return self._encoded_articles

    def articles_generator(self, batch_size = 1, start = 0, end = None):
        end = end or len(self.articles)

        for _ in range(batch_size - ((end - start - 1) % batch_size + 1)):
            yield self.EMPTY_ARTICLE

        for article in itertools.islice(self.encoded_articles, start, end):
            yield article

    def subbatch_generator(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.articles_generator, args=(batch_size, start, end), output_types=TYPE)
        dataset = dataset.padded_batch(batch_size, padded_shapes=([None]), drop_remainder=True)
        dataset = dataset.shuffle(100)

        for batch in dataset.as_numpy_iterator():
            remaining = batch
            while remaining.shape[1] > batch_length + 1:
                yield remaining[:, :batch_length + 1]
                remaining = remaining[:, batch_length:]

            if remaining.shape[1] == batch_length + 1:
                yield remaining
                yield np.zeros((batch_size, batch_length + 1), dtype=TYPE)
            else:
                yield np.hstack([remaining, np.zeros([batch_size, batch_length - remaining.shape[1] + 1])])

    def steps(self, batch_size, batch_length):
        articles = self.articles_generator(batch_size, batch_length)
        return sum(math.ceil(len(article) / batch_length + 1) for i, article in enumerate(articles) if (i + 1) % batch_size == 0)

    def dataset(self, batch_size, batch_length, start = 0, end = None):
        end = end or len(self.articles)

        dataset = tf.data.Dataset.from_generator(self.subbatch_generator, args=(batch_size, batch_length, start, end), output_types=TYPE, output_shapes=(batch_size, batch_length + 1))
        return dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

In [5]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_final_batch_ratio(true_labels, predictions):
    return 0 ** tf.math.abs(true_labels[-1, -1])

class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_epoch_begin(self, epoch, logs=None):
        self.last_final_batch_count = 0
        
    def on_batch_end(self, batch, logs={}):
        average_final_batch_ratio = logs.get('average_final_batch_ratio', 0)
        final_batch_count = int(round(average_final_batch_ratio * (batch + 1)))
        is_final = final_batch_count - self.last_final_batch_count
        self.last_final_batch_count = final_batch_count
        
        if is_final:
            self.model.reset_states()

class Model:
    def __init__(self, articles, checkpoint_dir, vocab_size, embedding_dim, rnn_units):
        self._articles = articles
        self._batch_size = None
        self._batched_item_length = None
        self._training_model = None
        self._predicting_model = None
        self._vocab_size = vocab_size
        self._embedding_dim = embedding_dim
        self._rnn_units = rnn_units

        self._checkpoint_dir = checkpoint_dir
        self._checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

    def training_model(self, batch_size, batched_item_length):
        if self._training_model == None or batch_size != self._batch_size or batched_item_length != self._batched_item_length:
            self._batch_size = batch_size
            self._batched_item_length = batched_item_length
            self._training_model = tf.keras.Sequential([
                checkpointable(tf.keras.layers.Masking)(mask_value=0, batch_input_shape=[batch_size, batched_item_length]),
                checkpointable(tf.keras.layers.Embedding)(self._vocab_size, self._embedding_dim),
                checkpointable(tf.keras.layers.GRU)(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                checkpointable(tf.keras.layers.GRU)(self._rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                checkpointable(tf.keras.layers.Dense)(self._vocab_size),
            ])

            if os.path.isdir(self._checkpoint_dir):
                self._training_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))

            self._training_model.compile(optimizer='adam', loss=loss, metrics=[average_final_batch_ratio])
            self._predicting_model = None
        
        return self._training_model

    @property
    def callbacks(self):
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self._checkpoint_prefix, save_weights_only=True)
        model_state_resetter_callback = ModelStateResetter()
        
        return [checkpoint_callback, model_state_resetter_callback]
    
    def train(self, batch_size, batched_item_length, epochs=1):
        dataset = self._articles.dataset(batch_size, batched_item_length)

        model = self.training_model(batch_size, batched_item_length)

        model.fit(dataset, epochs=epochs, callbacks=self.callbacks)
    
    @property
    def predicting_model(self):
        if self._predicting_model == None:
            self._predicting_model = tf.keras.Sequential([
                checkpointable(tf.keras.layers.Masking)(mask_value=0, batch_input_shape=[1, 1]),
                checkpointable(tf.keras.layers.Embedding)(self._vocab_size, self._embedding_dim),
                checkpointable(tf.keras.layers.GRU)(self._rnn_units, stateful=True, return_sequences=True),
                checkpointable(tf.keras.layers.GRU)(self._rnn_units, stateful=True, return_sequences=True),
                checkpointable(tf.keras.layers.Dense)(self._vocab_size),
            ])
            
            self._predicting_model.load_weights(tf.train.latest_checkpoint(self._checkpoint_dir))
            self._training_model = None
        
        return self._predicting_model
    
    def predict(self, input_eval):
        return self.predicting_model(input_eval)

In [6]:
articles = Articles('page_revisions_text_with_title')

In [7]:
model = Model(articles, './training_checkpoints-32', vocab_size = encoder.vocab_size, embedding_dim=128, rnn_units=512)

In [8]:
model.training_model(256, 256).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (256, 256)                0         
_________________________________________________________________
embedding (Embedding)        (256, 256, 128)           32640     
_________________________________________________________________
gru (GRU)                    (256, 256, 512)           986112    
_________________________________________________________________
gru_1 (GRU)                  (256, 256, 512)           1575936   
_________________________________________________________________
dense (Dense)                (256, 256, 255)           130815    
Total params: 2,725,503
Trainable params: 2,725,503
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.train(224, 224, epochs=10)

Epoch 1/10
1672/1672 [==============================] - 466s 279ms/step - loss: 2.2504 - average_final_batch_ratio: 0.1561
Epoch 2/10
1672/1672 [==============================] - 318s 190ms/step - loss: 1.5829 - average_final_batch_ratio: 0.1561
Epoch 3/10
1672/1672 [==============================] - 322s 193ms/step - loss: 1.4825 - average_final_batch_ratio: 0.1561
Epoch 4/10
1672/1672 [==============================] - 325s 194ms/step - loss: 1.4354 - average_final_batch_ratio: 0.1561
Epoch 5/10
1672/1672 [==============================] - 327s 196ms/step - loss: 1.4017 - average_final_batch_ratio: 0.1561
Epoch 6/10
1672/1672 [==============================] - 315s 189ms/step - loss: 1.3785 - average_final_batch_ratio: 0.1561
Epoch 7/10
1672/1672 [==============================] - 315s 188ms/step - loss: 1.3648 - average_final_batch_ratio: 0.1561
Epoch 8/10
1672/1672 [==============================] - 315s 189ms/step - loss: 1.3510 - average_final_batch_ratio: 0.1561
Epoch 9/10
1672/

In [9]:
import ctypes

class Huffman:
    huffman = ctypes.CDLL('x64/Release/huffman')
    
    huffman.create_tree.restype = ctypes.c_void_p
    huffman.destroy_tree.restype = None
    huffman.load_weights.restype = None
    huffman.create_code_string.restype = ctypes.c_char_p
    
    def __init__(self, category_count):
        self.category_count = category_count
        self.tree = ctypes.c_void_p(self.huffman.create_tree(category_count))

    def __del__(self):
        self.huffman.destroy_tree(self.tree)
        
    def load_weights(self, weights):
        self.huffman.load_weights(self.tree, weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float)))
    
    def get_code_length(self, category):
        return self.huffman.get_code_length(self.tree, category)

    def get_code_zero_count(self, category):
        return self.huffman.get_code_zero_count(self.tree, category)
    
    def archive_size(self, model, text):
        archived_size = math.ceil(math.log2(self.category_count))
        input_eval = np.array([[text[0]]], dtype=TYPE)

        model.predicting_model.reset_states()

        for byte in text[1:]:
            predictions = model.predict(input_eval)
            predictions = tf.squeeze(predictions, 0) # remove the batch dimension

            weights = tf.nn.softmax(predictions[0]).numpy()
            self.load_weights(weights)
            archived_size += self.get_code_length(byte.item())

            input_eval = tf.expand_dims([byte], 0)

        return archived_size

In [13]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.239583
Article 1000:	Length: 42	Avg Compression: 0.342593
Article 2000:	Length: 51	Avg Compression: 0.272619
Article 3000:	Length: 63	Avg Compression: 0.241071
Article 4000:	Length: 387	Avg Compression: 0.224324
Article 5000:	Length: 1254	Avg Compression: 0.225124
Article 6000:	Length: 2360	Avg Compression: 0.215309
Article 7000:	Length: 3684	Avg Compression: 0.230533
Article 8000:	Length: 5357	Avg Compression: 0.236913
Article 9000:	Length: 7861	Avg Compression: 0.226200
Article 10000:	Length: 11179	Avg Compression: 0.228717
Article 11000:	Length: 15815	Avg Compression: 0.221533
Article 12000:	Length: 25299	Avg Compression: 0.210647
Article 13000:	Length: 79403	Avg Compression: 0.219174


In [8]:
model.train(256, 256)

1339/1339 [==============================] - 450s 336ms/step - loss: 1.2619 - average_final_batch_ratio: 0.1650


In [9]:
model.train(256, 256, epochs=74)

Epoch 1/74
1339/1339 [==============================] - 308s 230ms/step - loss: 1.2569 - average_final_batch_ratio: 0.1650
Epoch 2/74
1339/1339 [==============================] - 313s 233ms/step - loss: 1.2504 - average_final_batch_ratio: 0.1650
Epoch 3/74
1339/1339 [==============================] - 311s 232ms/step - loss: 1.2468 - average_final_batch_ratio: 0.1650
Epoch 4/74
1339/1339 [==============================] - 318s 237ms/step - loss: 1.2448 - average_final_batch_ratio: 0.1650
Epoch 5/74
1339/1339 [==============================] - 314s 235ms/step - loss: 1.2411 - average_final_batch_ratio: 0.1650
Epoch 6/74
1339/1339 [==============================] - 316s 236ms/step - loss: 1.2354 - average_final_batch_ratio: 0.1650
Epoch 7/74
1339/1339 [==============================] - 317s 236ms/step - loss: 1.2343 - average_final_batch_ratio: 0.1650
Epoch 8/74
1339/1339 [==============================] - 311s 233ms/step - loss: 1.2308 - average_final_batch_ratio: 0.1650
Epoch 9/74
1339/

In [12]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.229167
Article 1000:	Length: 42	Avg Compression: 0.168981
Article 2000:	Length: 51	Avg Compression: 0.166667
Article 3000:	Length: 63	Avg Compression: 0.161458
Article 4000:	Length: 387	Avg Compression: 0.202027
Article 5000:	Length: 1254	Avg Compression: 0.212617
Article 6000:	Length: 2360	Avg Compression: 0.204635
Article 7000:	Length: 3684	Avg Compression: 0.218722
Article 8000:	Length: 5357	Avg Compression: 0.226779
Article 9000:	Length: 7861	Avg Compression: 0.216583
Article 10000:	Length: 11179	Avg Compression: 0.217647
Article 11000:	Length: 15815	Avg Compression: 0.212964
Article 12000:	Length: 25299	Avg Compression: 0.203061
Article 13000:	Length: 79403	Avg Compression: 0.210823


In [13]:
model.train(256, 256, epochs=15)

Epoch 1/15
1339/1339 [==============================] - 312s 233ms/step - loss: 1.1844 - average_final_batch_ratio: 0.1650
Epoch 2/15
1339/1339 [==============================] - 316s 236ms/step - loss: 1.1842 - average_final_batch_ratio: 0.1650
Epoch 3/15
1339/1339 [==============================] - 316s 236ms/step - loss: 1.1841 - average_final_batch_ratio: 0.1650
Epoch 4/15
1339/1339 [==============================] - 309s 231ms/step - loss: 1.1826 - average_final_batch_ratio: 0.1650
Epoch 5/15
1339/1339 [==============================] - 313s 234ms/step - loss: 1.1840 - average_final_batch_ratio: 0.1650
Epoch 6/15
1339/1339 [==============================] - 311s 232ms/step - loss: 1.1832 - average_final_batch_ratio: 0.1650
Epoch 7/15
1339/1339 [==============================] - 314s 235ms/step - loss: 1.1833 - average_final_batch_ratio: 0.1650
Epoch 8/15
1339/1339 [==============================] - 308s 230ms/step - loss: 1.1826 - average_final_batch_ratio: 0.1650
Epoch 9/15
1339/

In [14]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.239583
Article 1000:	Length: 42	Avg Compression: 0.159722
Article 2000:	Length: 51	Avg Compression: 0.154762
Article 3000:	Length: 63	Avg Compression: 0.160714
Article 4000:	Length: 387	Avg Compression: 0.207207
Article 5000:	Length: 1254	Avg Compression: 0.216211
Article 6000:	Length: 2360	Avg Compression: 0.205985
Article 7000:	Length: 3684	Avg Compression: 0.220903
Article 8000:	Length: 5357	Avg Compression: 0.228028
Article 9000:	Length: 7861	Avg Compression: 0.217930
Article 10000:	Length: 11179	Avg Compression: 0.220159
Article 11000:	Length: 15815	Avg Compression: 0.215378
Article 12000:	Length: 25299	Avg Compression: 0.204995
Article 13000:	Length: 79403	Avg Compression: 0.212611


In [8]:
model.training_model(512, 128).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (512, 128)                0         
_________________________________________________________________
embedding (Embedding)        (512, 128, 128)           32640     
_________________________________________________________________
gru (GRU)                    (512, 128, 512)           986112    
_________________________________________________________________
gru_1 (GRU)                  (512, 128, 512)           1575936   
_________________________________________________________________
dense (Dense)                (512, 128, 255)           130815    
Total params: 2,725,503
Trainable params: 2,725,503
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.train(512, 128, epochs=15)

Epoch 1/15
1602/1602 [==============================] - 489s 305ms/step - loss: 0.9856 - average_final_batch_ratio: 0.1011
Epoch 2/15
   1/1602 [..............................] - ETA: 17:25

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:GPU:0 to /job:localhost/replica:0/task:0/device:CPU:0 in order to run Identity: GPU sync failed [Op:Identity]

In [8]:
model.train(640, 128, epochs=4)

Epoch 1/4
1422/1422 [==============================] - 547s 385ms/step - loss: 0.8838 - average_final_batch_ratio: 0.0992
Epoch 2/4
1422/1422 [==============================] - 403s 283ms/step - loss: 0.8855 - average_final_batch_ratio: 0.0992
Epoch 3/4
1422/1422 [==============================] - 401s 282ms/step - loss: 0.8875 - average_final_batch_ratio: 0.0992
Epoch 4/4
1422/1422 [==============================] - 396s 278ms/step - loss: 0.8872 - average_final_batch_ratio: 0.0992


In [10]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.239583
Article 1000:	Length: 42	Avg Compression: 0.199074
Article 2000:	Length: 51	Avg Compression: 0.201190
Article 3000:	Length: 63	Avg Compression: 0.173363
Article 4000:	Length: 387	Avg Compression: 0.209234
Article 5000:	Length: 1254	Avg Compression: 0.216763
Article 6000:	Length: 2360	Avg Compression: 0.227932
Article 7000:	Length: 3684	Avg Compression: 0.234083
Article 8000:	Length: 5357	Avg Compression: 0.236289
Article 9000:	Length: 7861	Avg Compression: 0.221377
Article 10000:	Length: 11179	Avg Compression: 0.221105
Article 11000:	Length: 15815	Avg Compression: 0.215679
Article 12000:	Length: 25299	Avg Compression: 0.205101
Article 13000:	Length: 79403	Avg Compression: 0.211797


In [11]:
model.train(768, 128, epochs=5)

Epoch 1/5
1290/1290 [==============================] - 427s 331ms/step - loss: 0.8118 - average_final_batch_ratio: 0.0969
Epoch 2/5
1290/1290 [==============================] - 425s 329ms/step - loss: 0.8144 - average_final_batch_ratio: 0.0969
Epoch 3/5
1290/1290 [==============================] - 422s 327ms/step - loss: 0.9594 - average_final_batch_ratio: 0.0969
Epoch 4/5
1290/1290 [==============================] - 421s 327ms/step - loss: 1.0268 - average_final_batch_ratio: 0.0969
Epoch 5/5
1290/1290 [==============================] - 417s 323ms/step - loss: 1.0107 - average_final_batch_ratio: 0.0969


In [12]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.229167
Article 1000:	Length: 42	Avg Compression: 0.187500
Article 2000:	Length: 51	Avg Compression: 0.197619
Article 3000:	Length: 63	Avg Compression: 0.177083
Article 4000:	Length: 387	Avg Compression: 0.230856
Article 5000:	Length: 1254	Avg Compression: 0.243988
Article 6000:	Length: 2360	Avg Compression: 0.257436
Article 7000:	Length: 3684	Avg Compression: 0.267843
Article 8000:	Length: 5357	Avg Compression: 0.269938
Article 9000:	Length: 7861	Avg Compression: 0.253583
Article 10000:	Length: 11179	Avg Compression: 0.253112
Article 11000:	Length: 15815	Avg Compression: 0.249144
Article 12000:	Length: 25299	Avg Compression: 0.234899
Article 13000:	Length: 79403	Avg Compression: 0.241570


In [13]:
model.train(768, 128, epochs=15)

Epoch 1/15
1290/1290 [==============================] - 433s 335ms/step - loss: 0.9998 - average_final_batch_ratio: 0.0969
Epoch 2/15
1290/1290 [==============================] - 428s 332ms/step - loss: 0.9891 - average_final_batch_ratio: 0.0969
Epoch 3/15
1290/1290 [==============================] - 431s 334ms/step - loss: 0.9774 - average_final_batch_ratio: 0.0969
Epoch 4/15
1290/1290 [==============================] - 434s 337ms/step - loss: 0.9645 - average_final_batch_ratio: 0.0969
Epoch 5/15
1290/1290 [==============================] - 434s 337ms/step - loss: 0.9504 - average_final_batch_ratio: 0.0969
Epoch 6/15
1290/1290 [==============================] - 435s 337ms/step - loss: 0.9338 - average_final_batch_ratio: 0.0969
Epoch 7/15
1290/1290 [==============================] - 433s 336ms/step - loss: 0.9153 - average_final_batch_ratio: 0.0969
Epoch 8/15
1290/1290 [==============================] - 435s 337ms/step - loss: 0.8980 - average_final_batch_ratio: 0.0969
Epoch 9/15
1290/

In [14]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.270833
Article 1000:	Length: 42	Avg Compression: 0.182870
Article 2000:	Length: 51	Avg Compression: 0.194048
Article 3000:	Length: 63	Avg Compression: 0.168155
Article 4000:	Length: 387	Avg Compression: 0.200450
Article 5000:	Length: 1254	Avg Compression: 0.209577
Article 6000:	Length: 2360	Avg Compression: 0.221126
Article 7000:	Length: 3684	Avg Compression: 0.229562
Article 8000:	Length: 5357	Avg Compression: 0.231927
Article 9000:	Length: 7861	Avg Compression: 0.216993
Article 10000:	Length: 11179	Avg Compression: 0.217570
Article 11000:	Length: 15815	Avg Compression: 0.213926
Article 12000:	Length: 25299	Avg Compression: 0.203647
Article 13000:	Length: 79403	Avg Compression: 0.211110


## Нека запишем примерно предвиждане на данни; ще го използваме за тестване на кодиране на huffman и аритметично кодиране

In [33]:
len(articles.encoded_articles[5449])

1101

In [34]:
text = articles.encoded_articles[5449]
input_eval = np.array([[text[0]]], dtype=TYPE)

model.predicting_model.reset_states()

char_actual = []
char_predictions = []

for byte in text[1:]:
    predictions = model.predict(input_eval)
    predictions = tf.squeeze(predictions, 0) # remove the batch dimension

    weights = tf.nn.softmax(predictions[0]).numpy()
    char_actual.append(byte)
    char_predictions.append(weights)

    input_eval = tf.expand_dims([byte], 0)

In [35]:
with open('char-actual', 'wb') as file:
    file.write(np.array(char_actual).tobytes())

with open('char-predictions', 'wb') as file:
    file.write(np.array(char_predictions).tobytes())

## Да продължим с тренирането...

In [36]:
model.train(768, 128, epochs=15)

Epoch 1/15
1290/1290 [==============================] - 424s 329ms/step - loss: 0.8181 - average_final_batch_ratio: 0.0969
Epoch 2/15
1290/1290 [==============================] - 434s 336ms/step - loss: 0.8190 - average_final_batch_ratio: 0.0969
Epoch 3/15
1290/1290 [==============================] - 425s 330ms/step - loss: 0.8153 - average_final_batch_ratio: 0.0969
Epoch 4/15
1290/1290 [==============================] - 422s 327ms/step - loss: 0.8163 - average_final_batch_ratio: 0.0969
Epoch 5/15
1290/1290 [==============================] - 423s 328ms/step - loss: 0.8149 - average_final_batch_ratio: 0.0969
Epoch 6/15
1290/1290 [==============================] - 418s 324ms/step - loss: 0.8149 - average_final_batch_ratio: 0.0969
Epoch 7/15
1290/1290 [==============================] - 423s 328ms/step - loss: 0.8163 - average_final_batch_ratio: 0.0969
Epoch 8/15
1290/1290 [==============================] - 422s 327ms/step - loss: 0.8144 - average_final_batch_ratio: 0.0969
Epoch 9/15
1290/

In [37]:
total_raw = 0
total_compressed = 0

huffman = Huffman(encoder.vocab_size)
for index, encoded_article in enumerate(articles.articles_generator(1)):
    if index % 1000 == 0 and len(encoded_article) != 0:
        article = encoder.decode(encoded_article)
        total_raw += len(article) * 8
        total_compressed += huffman.archive_size(model, encoded_article)
        print('Article %d:\tLength: %d\tAvg Compression: %f' % (index, len(article), total_compressed/total_raw))

Article 0:	Length: 12	Avg Compression: 0.270833
Article 1000:	Length: 42	Avg Compression: 0.180556
Article 2000:	Length: 51	Avg Compression: 0.200000
Article 3000:	Length: 63	Avg Compression: 0.168899
Article 4000:	Length: 387	Avg Compression: 0.205180
Article 5000:	Length: 1254	Avg Compression: 0.217178
Article 6000:	Length: 2360	Avg Compression: 0.231500
Article 7000:	Length: 3684	Avg Compression: 0.237171
Article 8000:	Length: 5357	Avg Compression: 0.238579
Article 9000:	Length: 7861	Avg Compression: 0.224972
Article 10000:	Length: 11179	Avg Compression: 0.226946
Article 11000:	Length: 15815	Avg Compression: 0.221375
Article 12000:	Length: 25299	Avg Compression: 0.210802
Article 13000:	Length: 79403	Avg Compression: 0.217456


In [8]:
text = articles.encoded_articles[5449]

In [9]:
text

array([223,  32, 247, ..., 219, 184,  56], dtype=int16)

In [11]:
print(encoder.decode(text))

=bernoulli's inequality=
in [[mathematics]], '''bernoulli's inequality''' is an [[inequality]] that approximates [[exponentiation]]s of 1 X <i>x</i>.

the inequality states that
:<math>(1 X x)Xr \geq 1 X rx\!</math>
for every [[integer]] <i>r</i> X 0 and every [[real number]] <i>x</i> X X1. if the exponent <i>r</i> is [[even number|even]], then the inequality is valid for ''all'' real numbers <i>x</i>. the strict version of the inequality reads
:<math>(1 X x)Xr > 1 X rx\!</math>
for every integer <i>r</i> X 2 and every real number <i>x</i> X X1 with <i>x</i> X 0.

bernoulli's inequality is often used as the crucial step in the [[proof (math)|proof]] of other inequalities. it can itself be proved using [[mathematical induction]]. 

the exponent <i>r</i> can be generalized to an arbitrary real number as follows: if <i>x</i> > X1, then
:<math>(1 X x)Xr \geq 1 X rx\!</math>
for <i>r</i> X 0 or <i>r</i> X 1, and 
:<math>(1 X x)Xr \leq 1 X rx\!</math>
for 0 X <i>r</i> X 1.
this generalizatio

In [12]:
len(encoder.decode(text))

1653